# Install/Import Required Packages

In [1]:
!pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.1/481.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3


In [ ]:
from huggingface_hub import login
# login(token='**************** add_token_here ****************')

In [3]:
!pip install torch transformers datasets tqdm evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
!pip install accelerate

In [5]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.2 MB/s eta 0:00:00


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from tqdm import tqdm
import evaluate
import random

In [20]:
!pip install textattack
from textattack.augmentation import CharSwapAugmenter
from textattack.augmentation.augmenter import Augmenter
from textattack.transformations import WordSwapWordNet

# Model and Hardware Setup

In [38]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EXAMPLES = 1000  # ***** Temp Adjustment to 10 *****
MAX_NEW_TOKENS = 35

# Load and Quantize the LLM (Mistral-7B)

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Define Helper Methods

In [10]:
def format_prompt(article):
    return f"<s>[INST] Classify the following news article into one of these categories: World, Sports, Business, or Science/Technology.\n\nArticle: {article} [/INST]"

def extract_label(generated_text):
    answer = generated_text.split("[/INST]")[-1].strip().lower()

    if "world" in answer:
        return 0
    elif "sports" in answer:
        return 1
    elif "business" in answer:
        return 2
    elif "science" in answer or "technology" in answer:
        return 3
    else:
      return -1

# Load Dataset (AG News)

In [39]:
print("Loading AG News dataset...")
dataset = load_dataset("ag_news", split="test") # contains total of 7600 samples in test split
dataset = sorted(dataset, key=lambda x: len(x['text']))[:NUM_EXAMPLES]
print(f"Loaded {len(dataset)} shortest examples.")

Loading AG News dataset...
Loaded 1000 shortest examples.


# Define Adversarial Augmentation

In [40]:
adversarial_augmentation_enabled = True

augmenter = Augmenter(
    transformation=WordSwapWordNet(),
    pct_words_to_swap=0.2,
    transformations_per_example=1
)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Evaluate the Model

In [ ]:
y_true, y_pred = [], []
DEBUG = False
first_item = True

for item in tqdm(dataset):
    label = item["label"]
    article = item["text"]

    if DEBUG or first_item:
      print(f"\nOriginal article: {article}")

    # Apply WordSwapWordNet adversarial modification to the sentences
    if adversarial_augmentation_enabled:
      article = augmenter.augment(article)[0]

      if DEBUG or first_item:
        print(f"Adversarial article: {article}")

    prompt = format_prompt(article)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            pad_token_id=tokenizer.eos_token_id
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_label = extract_label(output_text)

    if DEBUG or first_item:
      output_text = output_text.split("[/INST]")[-1].strip().lower().replace('\n', ' ')
      print(f"\nOutput text: {output_text}")
      print(f"Actual label: {label}, Predicted label: {pred_label}")

    if pred_label != -1:
        y_true.append(label)
        y_pred.append(pred_label)

    first_item = False

  0%|          | 0/1000 [00:00<?, ?it/s]


Original article: Socially Responsible Funds on a Tear Don't be too impressed -- great returns don't always mean much.
Adversarial article: Socially Responsible Funds on a displume Don't be too impressed -- great returns don't always think much.


  0%|          | 1/1000 [00:00<15:07,  1.10it/s]


Output text: the article can be classified in the business category.
Actual label: 2, Predicted label: 2


100%|██████████| 1000/1000 [16:59<00:00,  1.02s/it]


# Results

In [42]:
accuracy = evaluate.load("accuracy")

def accuracy_score(y_true, y_pred):
  if len(y_true) and len(y_pred):
    return accuracy.compute(references=y_true, predictions=y_pred)["accuracy"]
  else:
    return 0

print("--- Evaluation Results ---")
print(f"\nAccuracy on AG News: {accuracy_score(y_true, y_pred) * 100:.2f}%")
print(f"\nNum of confident classifications: {len(y_pred)}")

--- Evaluation Results ---

Accuracy on AG News: 67.28%

Num of confident classifications: 975
